In [1]:
"""
train_human_gpt_small.py
------------------------
Train a small GPT-style model on your 12 MB human communication dataset.
Works on CPU or GPU.
"""

import os, torch, torch.nn as nn, torch.nn.functional as F
from tqdm import tqdm


In [2]:
pip show torch

Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: c:\users\aman\desktop\human-communication-gpt-with-1-gb-dataset\env\lib\site-packages
Requires: fsspec, networkx, jinja2, filelock, typing-extensions, sympy
Required-by: torchvision
Note: you may need to restart the kernel to use updated packages.


In [16]:
# ----------------------- Config -----------------------
DATA_PATH = "data/human_chat.txt"
MODEL_PATH = "checkpoints/human_gpt_small.pt"

BLOCK_SIZE = 256    # shorter context (small data)
BATCH_SIZE = 32       # reduce for limited VRAM
N_EMBD = 128
N_HEAD = 4
N_LAYER = 2
DROPOUT = 0.1
N_EMBD = 96       # smaller embedding
LR = 2e-4   
STEPS = 20000          # you can raise to 8000 for better quality

In [17]:
# ----------------------- Load Dataset -----------------------
print("📖 Loading dataset...")
text = open(DATA_PATH, encoding="utf-8").read()
print(f"✅ Loaded {len(text):,} characters from {DATA_PATH}")

import tiktoken
print("🧩 Using GPT-2 BPE tokenizer...")
enc = tiktoken.get_encoding("gpt2")

def encode(s): return enc.encode(s)
def decode(l): return enc.decode(l)

text = open(DATA_PATH, encoding="utf-8").read()
data = torch.tensor(encode(text), dtype=torch.long)
vocab_size = enc.n_vocab
print(f"🧩 Vocab size (BPE): {vocab_size}")

n = int(0.9 * len(data))
train_data, val_data = data[:n], data[n:]
print(f"📊 Tokens: train={len(train_data)}, val={len(val_data)}")

📖 Loading dataset...
✅ Loaded 12,950,920 characters from data/human_chat.txt
🧩 Using GPT-2 BPE tokenizer...


🧩 Vocab size (BPE): 50257
📊 Tokens: train=3238109, val=359790


In [18]:
# ----------------------- Model Definition -----------------------
class MiniGPT(nn.Module):
    def __init__(self, vocab_size, n_embd=N_EMBD, n_head=N_HEAD,
                 n_layer=N_LAYER, block_size=BLOCK_SIZE, dropout=DROPOUT):
        super().__init__()

        # Embeddings
        self.token_emb = nn.Embedding(vocab_size, n_embd)
        self.pos_emb = nn.Embedding(block_size, n_embd)

        # Transformer blocks
        self.blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=n_embd,
                nhead=n_head,
                dim_feedforward=4 * n_embd,
                dropout=dropout,
                batch_first=True
            )
            for _ in range(n_layer)
        ])

        # Final normalization + dropout + output head
        self.ln_f = nn.LayerNorm(n_embd)
        self.dropout = nn.Dropout(0.1)         # ✅ add this line
        self.head = nn.Linear(n_embd, vocab_size)
        self.block_size = block_size

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_emb(idx)
        pos_emb = self.pos_emb(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb

        # pass through each transformer block
        for block in self.blocks:
            x = block(x)

        # final layer norm + dropout
        x = self.ln_f(x)
        x = self.dropout(x)                    # ✅ add this line

        logits = self.head(x)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss


In [19]:
# ----------------------- Training -----------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Using device: {device}")
model = MiniGPT(vocab_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

def get_batch(split):
    data_split = train_data if split == "train" else val_data
    ix = torch.randint(len(data_split) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data_split[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data_split[i+1:i+BLOCK_SIZE+1] for i in ix])
    return x.to(device), y.to(device)

print("🧠 Starting training...")
for step in tqdm(range(STEPS)):
    xb, yb = get_batch("train")
    _, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if step % 500 == 0 or step == STEPS - 1:
        print(f"Step {step:05d} | Loss: {loss.item():.4f}")


🚀 Using device: cuda
🧠 Starting training...


  0%|          | 3/20000 [00:00<1:06:02,  5.05it/s]

Step 00000 | Loss: 11.0007


  3%|▎         | 503/20000 [00:52<31:47, 10.22it/s]

Step 00500 | Loss: 5.4459


...

Step 19000 | Loss: 0.0315


 98%|█████████▊| 19503/20000 [34:08<00:49, 10.14it/s]

Step 19500 | Loss: 0.0355


100%|██████████| 20000/20000 [35:00<00:00,  9.52it/s]

Step 19999 | Loss: 0.0380


In [22]:
# ==========================
# 💬 TEXT GENERATION
# ==========================
def generate(model, start="User: Hello!\nBot:", max_new_tokens=150, temperature=0.9):
    model.eval()
    idx = torch.tensor([encode(start)], dtype=torch.long).to(device)
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -model.block_size:]
        logits, _ = model(idx_cond)
        logits = logits[:, -1, :] / temperature
        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, next_id), dim=1)
    return decode(idx[0].tolist())

In [23]:

# ==========================
# 🤖 TEST CHAT GENERATION
# ==========================
prompt = """
User1: You never reply to my messages.
User2: I was busy with work.
Please help me communicate better.
Bot:
"""

print("\n================== GENERATED RESPONSE ==================\n")
print(generate(model, start=prompt, temperature=0.7, max_new_tokens=150))
print("\n========================================================")


================== GENERATED RESPONSE ==================


User1: You never reply to my messages.
User2: I was busy with work.
Please help me communicate better.
Bot:

 never crashed help me. I to catch my car. Somh po. Generally crashed me true
 SC. Ifh po If slamming sometime
I gave po If me  po If po If If po po. If Always help If my plea shown. peaceful!! better spill to my work busy wanted work my preferences me1est. po true! If If embarrassed po trust catch trust tastes never busy! Cheap po rocks If po say po true If If true embarrassed po po repeat If there properties freeze coaching. If proceed po owes me out. po If trust true po If embarrassed callates tastes true po If rent gre true.. If If helping flooded coaching. If true! If proceed rocked shower po po. Senior cardio prank hack true DOES slamming

